##  加载数据

### 1)定义输入数据并预处理数据。这里,我们首先读取数据 MNIST,并分别得到训练集的图片和标记的矩阵,以及测试集的图片和标记的矩阵

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./MNIST_data',one_hot=True)
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

/home/mzl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


### 接着,需要处理输入的数据,把上述 trX 和 teX 的形状变为[-1,28,28,1],-1 表示不格虑输入图片的数量,28×28 是图片的长和宽的像素数,1 是通道(channel)数量,因为 MNIST 的图片是黑白的,所以通道是 1,如果是 RGB 彩色图像,通道是 3

In [4]:
trX = trX.reshape(-1,28,28,1) # 28*28*1 input img
teX = teX.reshape(-1,28,28,1) # 28*28*1 input img
X = tf.placeholder("float",[None,28,28,1])
Y = tf.placeholder("float",[None,10])

### 2)初始化权重与定义格络结构。这里,我们将要构建一个拥有 3 个卷积层和 3 个池化层,随后接 1 个全连接层和 1 个输出层的卷积神经格络。首先定义初始化权重的函数

In [5]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))

### 初始化权重格法如下,我们设置卷积核的大小为 3×3:

In [18]:
w = init_weights([3,3,1,32]) # patch 大小为 3 × 3，输入维度为1,输出维度为32
w2 = init_weights([3,3,32,64]) # patch 大小为 3 × 3 输入维度为32,输出维度为64
w3 = init_weights([3,3,64,128]) # patch 大小为 3 × 3，输入维度为 64,输出维度为128
w4 = init_weights([128*4*4,625]) # 全链接层，输入维度为128×4×4，是上一层的输入数据由三维转换成一维,输出维度为 625
w_o = init_weights([625,10]) # 输出层，输入维度为625,输出维度为10,代表10类（labels）

### 随后,定义一个模型函数,代码如下:

In [19]:
# 神经网络模型构建函数，传入参数
# X：输入数据
# w: 每一层的权重
# p_keep_conv,p_keep_hidden:droupt 要保留神经元的比例
def model(X,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hidden):
    # 第一组卷积层及池化层，最后 droupt 一些神经元
    l1a = tf.nn.relu(tf.nn.conv2d(X,w,strides= [1,1,1,1],padding = 'SAME'))
    # l1a shape = (?,28,28,32)
    l1 = tf.nn.max_pool(l1a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    # l1 shape = (?,14,14,32)
    l1 = tf.nn.dropout(l1,p_keep_conv)
    
    #第二组卷积层及池化层，最后 droupt 一些神经元
    l2a = tf.nn.relu(tf.nn.conv2d(l1,w2,strides=[1,1,1,1],padding='SAME'))
    # L2a shape = (?,14,14,64)
    l2 = tf.nn.max_pool(l2a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    # 12 shaep = (?,7,7,64)
    l2 = tf.nn.dropout(l2,p_keep_conv)
    
    #第三组卷积及池化层，最后 droupt 一些神经元
    l3a = tf.nn.relu(tf.nn.conv2d(l2,w3,strides=[1,1,1,1],padding='SAME'))
    # l3a shape = (?,7,7,128)
    l3 = tf.nn.max_pool(l3a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    # l3 shape = (?,4,4,128)
    l3 = tf.reshape(l3,[-1,w4.get_shape().as_list()[0]]) # reshape to (?,2048)
    l3 = tf.nn.dropout(l3,p_keep_conv)
    
    #全链接层，最后 droupt 一些神经元
    l4 = tf.nn.relu(tf.matmul(l3,w4))
    l4 = tf.nn.dropout(l4,p_keep_hidden)
    
    #输出层
    pyx = tf.matmul(l4,w_o)
    return pyx # 返回预测值

### 我们定义 dropout 的占位符—keep_conv,它表示在一层中有多少比例的神经元被保留下来。生成格络模型,得到预测值,如下:

In [20]:
p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hidden) # 得到预测值

### 定义损失函数,这里采用 tf.nn.softmax_cross_entropy_with_logits 来比较预测值和真实值的差异,并做均值处理;定义训练的操作(train_op),采用实现 RMSProp 算法的优化器 tf.train.RMSPropOptimizer,学习率为 0.001,衰减值为 0.9,使损失最小;定义预测的操作(predict_op)

In [24]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.argmax(py_x,1)

## 训练模型和评估模型

In [25]:
batch_size = 64
test_size = 256

In [27]:
#Launch the graph in a session
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        training_batch = zip(range(0,len(trX),batch_size),
                             range(batch_size,len(trX)+1,batch_size))
        for start,end in training_batch:
            sess.run(train_op,feed_dict={X:trX[start:end],Y:trY[start:end],
                                        p_keep_conv:0.8,p_keep_hidden:0.5})
        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        print(i,np.mean(np.argmax(teY[test_indices],axis=1) ==
                       sess.run(predict_op,feed_dict={X:teX[test_indices],
                                                     p_keep_conv:1.0,
                                                     p_keep_hidden:1.0})))

0 0.94921875
1 0.984375
2 0.98828125
3 0.984375
4 0.98828125
5 0.9921875
6 1.0
7 0.99609375
8 0.9921875
9 1.0
10 0.9921875
11 0.99609375
12 0.984375
13 0.984375
14 0.99609375
15 0.9765625
16 0.99609375
17 0.9921875
18 0.984375
19 0.984375
20 0.984375
21 0.98828125
22 0.96484375
23 0.9921875
24 0.98046875
25 1.0
26 0.99609375
27 0.99609375
28 0.984375
29 0.99609375
30 0.9921875
31 0.96484375
32 0.99609375
33 0.9921875
34 0.98828125
35 0.99609375
36 0.98828125
37 0.984375
38 0.9921875
39 0.9921875
40 0.9921875
41 0.99609375
42 0.984375
43 0.9921875
44 0.9921875
45 0.9921875
46 0.9921875
47 0.99609375
48 0.98828125
49 0.99609375
50 0.984375
51 0.98046875
52 0.98046875
53 0.9921875
54 0.9921875
55 0.98046875
56 0.99609375
57 0.984375
58 0.98828125
59 0.97265625
60 0.98828125
61 0.9921875
62 0.99609375
63 0.984375
64 0.9765625
65 0.98046875
66 0.97265625
67 0.9765625
68 0.9921875
69 0.98828125
70 0.98828125
71 0.97265625
72 0.99609375
73 0.9765625
74 0.98828125
75 0.98828125
76 0.98046875
7